In [3]:
import pandas as pd
import numpy as np
import scipy as sp

from scipy import stats

def get_earnings(f_size):
    r = min(2, f_size)
    rest = f_size - r
    return r * 800 + np.random.normal(0, 100, 1)[0] + rest * 250

<h2>Отношение R</h2>
<h4>Сгенерируем данные выборки для примера</h4>
Пусть N = 1000, n = 100

In [4]:
fams = sp.random.poisson(1.5, 100) + 1
fams

array([4, 1, 3, 5, 3, 1, 5, 1, 3, 1, 1, 3, 3, 3, 1, 1, 1, 3, 1, 4, 3, 2,
       2, 3, 4, 3, 3, 3, 3, 2, 3, 2, 1, 2, 4, 2, 2, 6, 1, 1, 2, 4, 4, 1,
       3, 3, 2, 2, 1, 1, 1, 2, 4, 2, 2, 2, 2, 3, 1, 1, 3, 1, 4, 4, 4, 4,
       2, 2, 1, 3, 3, 2, 4, 1, 1, 3, 2, 3, 3, 1, 1, 2, 2, 2, 1, 2, 2, 4,
       3, 3, 2, 3, 1, 4, 2, 3, 4, 2, 2, 3])

In [5]:
df = pd.DataFrame([(f, get_earnings(f)) for f in fams], columns=['family_size', 'total_income'])
df

,family_size,total_income
0,4,2128.903264
1,1,805.089514
2,3,1653.050740
3,5,2424.817631
4,3,1894.210987
...,...,...
95,3,2034.599235
96,4,2189.017396
97,2,1642.255980
98,2,1649.762056


<h4>Найдем отношение R: общий заработок / количество человек в семье</h4>

In [6]:
R = df['total_income'].sum()/df['family_size'].sum()
R

640.5458431653183

Определим основные переменные

In [7]:
N=1000
n=len(df)
f=n/N
N, n, f

(1000, 100, 0.1)

In [8]:
x = df['family_size'].values
y = df['total_income'].values

In [9]:
x

array([4, 1, 3, 5, 3, 1, 5, 1, 3, 1, 1, 3, 3, 3, 1, 1, 1, 3, 1, 4, 3, 2,
       2, 3, 4, 3, 3, 3, 3, 2, 3, 2, 1, 2, 4, 2, 2, 6, 1, 1, 2, 4, 4, 1,
       3, 3, 2, 2, 1, 1, 1, 2, 4, 2, 2, 2, 2, 3, 1, 1, 3, 1, 4, 4, 4, 4,
       2, 2, 1, 3, 3, 2, 4, 1, 1, 3, 2, 3, 3, 1, 1, 2, 2, 2, 1, 2, 2, 4,
       3, 3, 2, 3, 1, 4, 2, 3, 4, 2, 2, 3])

In [10]:
y

array([2128.90326401,  805.08951364, 1653.05073965, 2424.81763113,
       1894.21098687,  832.74601171, 2388.90854537,  843.79139591,
       1810.08086132,  875.08883058,  764.65583262, 1901.38563139,
       1690.44461052, 1708.71591809,  758.35278526,  900.20349051,
        802.50116397, 1921.94118813,  880.1112167 , 2025.68586245,
       1668.21331455, 1757.17940164, 1571.81451383, 1828.09409555,
       1965.25487564, 1908.09397015, 1766.45958602, 1738.25645727,
       1734.02321742, 1630.95389839, 1845.20820055, 1563.77273658,
        951.4757296 , 1615.22102908, 2083.21658076, 1466.77855279,
       1509.10971713, 2569.52078176,  754.91215159,  866.27933486,
       1722.74080602, 2061.61002455, 2211.0226249 ,  902.7112208 ,
       1996.21820375, 1898.31548858, 1610.88841209, 1392.87298234,
        871.07507433,  728.11080509,  888.60023441, 1605.72333987,
       2035.93735078, 1508.29770097, 1536.44311327, 1500.53028442,
       1630.62806713, 1881.18269714,  753.24120328,  631.30497

Выборочная оценка R (в данном случае равна R)

In [11]:
R_o = y.mean()/x.mean()
R_o, R

(640.5458431653183, 640.5458431653183)

<h4>Посчитаем стандартную ошибку отношения R</h4>
Сперва дисперсия s2

In [12]:
s2 = ((y*y).sum() - 2*R*(x*y).sum() + (R**2)*(x*x).sum()) / (n-1)
s2

107449.10659735673

In [13]:
std = np.sqrt(s2)
std

327.7943053156304

Теперь стандартную ошибку s (с учетом поправки на конечность совокупности)

In [15]:
s = std * (np.sqrt(1-f))/(np.sqrt(n)*x.mean())
s

12.850123249959063

<b>95% Доверительный интервал</b> для отношения <br>
для больших n можно использовать нормальную аппроксимацию следующим образом:

In [16]:
perc = 0.95
alpha = 1 - perc
t = sp.stats.norm.ppf(1-alpha/2)

Rl = R_o - t*s
Rr = R_o + t*s
Rl,Rr

(615.3600643984977, 665.7316219321389)

<h2>Доли p</h2><br>
Для предыдущего примера посчитаем <b>долю</b> семей, которые состоят из <b>более двух чловек</b>

In [17]:
df1 = df[df['family_size']>2]
df1

,family_size,total_income
0,4,2128.903264
2,3,1653.050740
3,5,2424.817631
4,3,1894.210987
6,5,2388.908545
8,3,1810.080861
11,3,1901.385631
12,3,1690.444611
13,3,1708.715918
17,3,1921.941188


Вычислим долю p, а также q = 1 - p

In [18]:
a = len(df1)
p = a/n
q = 1 - p
p,q

(0.46, 0.54)

Посчитаем 95% <b>доверительный интервал</b> для доли p <br>

In [19]:
perc = 0.95
alpha = 1 - perc
t = sp.stats.norm.ppf(1-alpha/2)
t

1.959963984540054

In [20]:
ci_R = p + (t*np.sqrt(1-f)*np.sqrt((p*q)/(n-1)) + 1/(2*n))
ci_L = p - (t*np.sqrt(1-f)*np.sqrt((p*q)/(n-1)) + 1/(2*n))
ci_L, ci_R

(0.36186186353493194, 0.5581381364650682)

Если хотим получить оценку количества элементов в генеральной совокупности, домножаем на N

In [47]:
N*ci_L, N*ci_R

(313.08853624147014, 506.9114637585298)

Эти значения показывают, что в генеральной совокупности размера N=1000 количество семей из <b>болле двух человек</b> с 95% вероятностью будет равно значению из данного интервала

In [33]:
df2 = pd.read_csv('data/les3_data_example.csv')
df2

,col1,col2,col3,col4
0,52,30,1,2
1,36,19,1,1
2,56,30,4,2
3,74,40,1,3
4,42,19,1,2
...,...,...,...,...
6157,49,29,1,2
6158,35,14,1,1
6159,29,27,1,1
6160,38,21,1,1


In [43]:
N=10000
n=len(df2)
f = n/N

<h3>Найти:</h3>
    отношение R (col2/col4)<br>
    стандартную ошибку отношения<br>
    долю p для col3=1<br>
    95% ДИ для p<br>

In [37]:
y = df2['col2'].values
x = df2['col4'].values

R = y.mean() / x.mean()

In [38]:
s2 = ((y*y).sum() - 2*R*(x*y).sum() + (R**2)*(x*x).sum()) / (n-1)
s2

257.9565536282697

In [40]:
std = np.sqrt(s2)
std

16.061025920789422

In [44]:
s = std * (np.sqrt(1-f))/(np.sqrt(n)*x.mean())
s

0.06558602646554632

In [45]:
df3 = df2[df2['col3']==1]
df3

,col1,col2,col3,col4
0,52,30,1,2
1,36,19,1,1
3,74,40,1,3
4,42,19,1,2
5,32,18,1,1
...,...,...,...,...
6157,49,29,1,2
6158,35,14,1,1
6159,29,27,1,1
6160,38,21,1,1


In [46]:
a = len(df3)
p = a/n
q = 1 - p
p,q

(0.6335605322947095, 0.36643946770529046)

In [47]:
perc = 0.95
alpha = 1 - perc
t = sp.stats.norm.ppf(1-alpha/2)
t

1.959963984540054

In [48]:
ci_R = p + (t*np.sqrt(1-f)*np.sqrt((p*q)/(n-1)) + 1/(2*n))
ci_L = p - (t*np.sqrt(1-f)*np.sqrt((p*q)/(n-1)) + 1/(2*n))
ci_L, ci_R

(0.6260257199797205, 0.6410953446096986)

<h2>Задача отношения-R-1</h2>

In [51]:
def calc_ci(N, n, y_sum, x_sum, y2_sum, x2_sum, yx_sum):
    f = n/N
    x_mean = x_sum/n
    y_mean = y_sum/n
    R = y_mean / x_mean
    
    print("R: ", R)
    s2 = (y2_sum - 2*R*yx_sum + (R**2)*x2_sum) / (n-1)
    print("s2: ", s2)
    std = np.sqrt(s2)
    s = std * (np.sqrt(1-f))/(np.sqrt(n)*x_mean)
    print("s:", s)
    
    perc = 0.90
    alpha = 1 - perc
    t = sp.stats.norm.ppf(1-alpha/2)

    Rl = R - t*s
    Rr = R + t*s
    print(Rl,Rr)
    return R, s2, s, Rl, Rr

In [52]:
N = 468
n = 100
n_gos = 54
n_priv = 46

In [53]:
calc_ci(
    N=N,
    n=n_gos,
    y_sum=31281,
    x_sum=2024,
    y2_sum=29881219,
    x2_sum=111090,
    yx_sum=1729349
    )

R:  15.455039525691701
s2:  55880.5157168438
s: 0.8072235406671366
14.127274957064753 16.78280409431865


(15.455039525691701,
 55880.5157168438,
 0.8072235406671366,
 14.127274957064753,
 16.78280409431865)

In [54]:
R, s2, s, Rl, Rr = calc_ci(
    N=N,
    n=n_priv,
    y_sum=13707,
    x_sum=1075,
    y2_sum=6366785,
    x2_sum=33119,
    yx_sum=431041
    )

R:  12.750697674418603
s2:  16869.66024841774
s: 0.7781385270756288
11.47077369588758 14.030621652949627


In [64]:
N = 251
n = n_gos
y_summ = 2024
y_summ_2 = 111090
perc = 0.90
f = n / N

In [58]:
y_mean = y_summ/n
Y_summ = y_mean*N
y_mean, Y_summ

(37.48148148148148, 9407.851851851852)

In [60]:
var = (y_summ_2 - np.power(y_summ,2)/n) / (n-1)
var

664.6694619147448

In [61]:
std = np.sqrt(var)
std

25.781184261293056

In [62]:
std_mean_error = (std * np.sqrt(1-f)) / np.sqrt(n)
std_mean_error

3.108151900008348

In [65]:
alpha = 1 - perc
t = sp.stats.norm.ppf(1-alpha/2)
t

1.6448536269514722

In [66]:
Y_mean_left =y_mean - t * std_mean_error
Y_mean_right =y_mean + t * std_mean_error
Y_mean_left, Y_mean_right

(32.36902655563664, 42.593936407326325)

In [67]:
N*Y_mean_left, N*Y_mean_right

(8124.625665464796, 10691.078038238908)

<h2>Задача Доли-p-1</h2>

In [68]:
N=2000
n=200
f = n/N
a = 120
p = a/n
q = 1 - p
p,q

(0.6, 0.4)

In [69]:
perc = 0.95
alpha = 1 - perc
t = sp.stats.norm.ppf(1-alpha/2)
t

1.959963984540054

In [71]:
ci_R = p + (t*np.sqrt(1-f)*np.sqrt((p*q)/(n-1)) + 1/(2*n))
ci_L = p - (t*np.sqrt(1-f)*np.sqrt((p*q)/(n-1)) + 1/(2*n))
N*ci_L, N*ci_R

(1065.8547539462252, 1334.1452460537748)

In [72]:
a = 57
p = a/n
q = 1 - p
p,q
ci_R = p + (t*np.sqrt(1-f)*np.sqrt((p*q)/(n-1)) + 1/(2*n))
ci_L = p - (t*np.sqrt(1-f)*np.sqrt((p*q)/(n-1)) + 1/(2*n))
N*ci_L, N*ci_R

(445.9996457867481, 694.0003542132517)

In [73]:
a = 23
p = a/n
q = 1 - p
p,q
ci_R = p + (t*np.sqrt(1-f)*np.sqrt((p*q)/(n-1)) + 1/(2*n))
ci_L = p - (t*np.sqrt(1-f)*np.sqrt((p*q)/(n-1)) + 1/(2*n))
N*ci_L, N*ci_R

(140.90051066084806, 319.099489339152)